In [3]:
# default_exp core

# module name here

> API details.

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

[nb_2_html](https://github.com/BNIA/dataplay/blob/main/notebooks/04_nb_2_html.ipynb)

In [6]:
#export
import re
from IPython.display import clear_output
import marko

In [7]:
#export
from nbdev import export2html
from nbdev.export2html import read_nb, get_metadata, clean_exports
from nbdev.export import split_flags_and_code, check_re_multi

In [8]:
#export
# 1st fn. passes each cell to decision fn.
def convertNb(nb):  return [cleanCell(c) for c in nb['cells']]

# 2nd fn. returns text or passes cell to 'code cell' processor
def cleanCell(cell):
  if cell.cell_type == 'markdown':
    source = cell.source # if ( re.search('<details><summary>', source) ): return source.replace('\r\n', "source")
    return marko.convert(source)
  else: return processCode(cell)

# 3rd fn. Calls getFlags, processSource, processOutput
def processCode(cell):
  x = [cell.source]
  flags = getFlags(x[0])
  x[0] = processSource( x[0], flags )
  if ( len(cell.outputs) ):
    for o in cell.outputs: x.append( processOutput(o, flags) )
  return x

In [9]:
#export
# Detect and stripout and handle flags.
def getFlags(source):
  input_aug = ['#collapse_input_open', '#collapse_input', '#collapse_output_open', '#collapse_output',
               '#hide_input', '#hide_output', '#hide ', '%%capture', '%%javascript', '%%html']
  return [ x for x in input_aug if re.search(x, source) ]

# enclose the text within a collapsable html drawer.
def makeDetails( content, open ):
  return "<details "+('open' if open else '')+"> <summary>Click to toggle</summary> "+content+"</details>"

In [10]:
#export
# Strip Flags from text, make details, hide all.
def processSource (source, flags):
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_input_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_input'): source = makeDetails( source, False )
    if (lbl == '#hide '): source = ''
    if (lbl == '#hide_input'): source = ''
    if (lbl == '%%javascript'): source = ''
    if (lbl == '%%html'): source = ''
    if (lbl == '%%capture'): source = ''
  return source

In [67]:
#export
def processOutput(source, flags):
  if(source.output_type=='error'):  return ''
  if(source.output_type=='stream'):
    if(source.name=='stderr'): return ''
    source.data = {'text/html': source.text} # This will have the stream process as text/html.
  print(source.data.keys())
  if ( hasattr(source.data, 'text/html') ): source=source.data['text/html'];
  elif ( hasattr(source.data, 'application/javascript') ): source='<script>'+source.data['application/javascript']+'</script>'
  elif ( hasattr(source.data, 'image/png') ): source= "<img src=\"data:image/png;base64," + source.data['image/png'] + "\" alt='Image Alt Text'>"
  elif ( hasattr(source.data, 'text/plain') ): source = '' if re.search("<Figure", source.data['text/plain']) else source.data['text/plain'];
  # print(source)
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_output_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_output'): source = makeDetails( source, False )
    if (lbl == '#hide_output'): source = ''
    if (lbl == '#hide '): source = ''
  return source
  #output_type == 'stream' ==> text
  #output_type == 'display_data' ==> data{'application/javascript' or 'text/html' or 'execute_result'}

In [64]:
ls

000_core.ipynb  001_core_test.ipynb  _devserver.ipynb  index.ipynb  test.html


In [66]:
nb = read_nb('index.ipynb')
outp = convertNb(nb)

dict_keys(['image/png', 'text/plain'])
dict_keys(['text/plain'])
dict_keys(['image/png', 'text/plain'])
dict_keys(['image/png', 'text/plain'])
dict_keys(['image/png', 'text/plain'])


In [50]:
#export
def runit(fname='index.ipynb'): 
  nb = read_nb(fname)
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  txt = ''.join( [''.join(c) for c in outp] )
  return txt

In [52]:
t = runit()